In [ ]:
!kaggle datasets download -d vivek468/beginner-chest-xray-image-classification

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import zipfile



In [ ]:
# ZIP dosyasının yolu
zip_file_path = '/content/beginner-chest-xray-image-classification.zip'

# Çıkartılacak hedef klasör
extracted_folder_path = '/content/'

# ZIP dosyasını aç
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)




In [ ]:
# Veri setinin dizini
data_dir = "/content/chest_xray"

# Veri ön işleme ve artırma
data_transforms = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
])

# Veri yükleyicileri oluşturma
train_dataset = datasets.ImageFolder(root=data_dir + '/train', transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)

test_dataset = datasets.ImageFolder(root=data_dir + '/test', transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4)



In [ ]:
!pip install timm

import timm

# Xception modelini yükleme
xception_model = timm.create_model('xception', pretrained=True)

# Son fully connected layer'ı değiştirme
num_ftrs = xception_model.get_classifier().in_features
xception_model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# Eğitim için CUDA kullanılabilirse GPU'ya taşıma
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xception_model = xception_model.to(device)

optimizer = optim.SGD(xception_model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()



In [ ]:
from tqdm import tqdm

epochs = 1
for epoch in range(epochs):
    running_loss = 0.0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}", unit="batch", colour='green', leave=True)
    for batch_idx, (inputs, labels) in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        # Gradientleri sıfırlama
        optimizer.zero_grad()

        # Forward pass
        outputs = xception_model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass ve optimizasyon
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        progress_bar.set_postfix({'Loss': loss.item()})

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")



In [ ]:
correct = 0
total = 0
TP = 0
FP = 0
FN = 0
TN = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = xception_model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Kesinlik ve Duyarlılık için TP, FP, FN ve TN değerlerini hesapla
        TP += ((predicted == labels) & (labels == 1)).sum().item()
        FP += ((predicted != labels) & (labels == 0)).sum().item()
        FN += ((predicted != labels) & (labels == 1)).sum().item()
        TN += ((predicted == labels) & (labels == 0)).sum().item()

# Doğruluk (Accuracy) hesapla
accuracy = correct / total

# Kesinlik (Precision) hesapla
precision = TP / (TP + FP) if (TP + FP) != 0 else 0

# Duyarlılık (Recall) hesapla
recall = TP / (TP + FN) if (TP + FN) != 0 else 0

# F1-Skoru hesapla
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")

print(f"TP: {TP}")
print(f"FP: {FP}")
print(f"FN: {FN}")
print(f"TN: {TN}")